In [19]:
suppressMessages(library(rwwa))

varnm <- "rx3day"
nsamp <- 1000

mdl_args <- list("rx3day" = list(dist = "gev", type = "fixeddisp"),
                 "pr-jjaso" = list(dist = "norm", type = "fixeddisp"))[[varnm]]
#

rp <- switch(varnm, "rx3day" = 10, "pr-jjaso" = 10)

png_res <- 240

# GMST only

## Obs

In [2]:
gmst <- read.table("ts-obs/gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2025]

cov_2025 <- gmst[gmst$year == 2025,c("gmst"),drop = F]
cov_cf <- cov_2025 - 1.3

In [ ]:
fl <- list.files("ts-obs", pattern = paste0("mexico-floods_",varnm), full.names = T)
for (fnm in fl) {

    ds_nm = gsub(".+jjaso_|.dat","",fnm)
    res_fnm = paste0("res-obs/res-obs_",varnm,"_gmst-only_",ds_nm,".csv")
    
    df <- merge(gmst, load_ts(fnm, col.names = c("year", "x")))
    mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = "gmst", lower = F)))

    if (ds_nm %in% c("chirps", "mxhighresclimdb")) {
        rl_2025 <- eff_return_level(mdl, rp = rp, fixed_cov = cov_2025)
        mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = "gmst", lower = F,
                                                     ev = rl_2025, ev_year = 2025)))
    }

    if (file.exists(res_fnm)) next
    boot_res <- boot_ci(mdl, cov_f = cov_2025, cov_cf = cov_cf, nsamp = 1000)
    write.csv(boot_res, res_fnm)

    plot_fnm <- gsub("csv", "png", gsub("res-obs/res-obs", "fig/trendplots", res_fnm))
    png(plot_fnm, h = png_res, w = 3 * png_res*1.5); {
        prep_window(c(1,3))
        plot_trend(mdl)
        plot_covtrend(mdl, "gmst")
        plot_returnlevels(mdl, cov_f = cov_2025, cov_cf = cov_cf[1,,drop = F], nsamp = 50, legend_pos = "bottomright")
    }; dev.off()
}

## CORDEX

In [ ]:
for (fnm in list.files("ts-cordex", pattern = paste0(varnm,"_CAM"), full.names = T)) {

    gcm <- gsub("_", "_rcp85_", gsub("i1p1","",paste0(strsplit(fnm, "_")[[1]][4:5], collapse = "_")))
    mdl <- gsub(".dat","",paste0(strsplit(fnm, "_")[[1]][4:6], collapse = "_"))
    print(mdl)
    # if (mdl %in% c("CNRM-CM5_r1i1p1_ALADIN53","HadGEM2-ES_r1i1p1_HadREM3-GA7-05")) next
                
    res_fnm <- gsub(".+/", "res-mdl-gmstonly/res-cordex_", gsub(".dat", ".csv", fnm))
    if (file.exists(res_fnm)) next
    
    gmst_fnm <- list.files("~/00_WWA_project_folder/live/data/cmip5_gmsts/",
                           pattern = paste0(gcm), full.names = T)
    if(length(gmst_fnm) == 0) next

    # load all the data
    ts <- load_ts(fnm, col.names = c("year", "x"))
    gmst <- load_ts(gmst_fnm, col.names = c("year", "gmst"))
    df <- merge(gmst, ts)
    df <- df[df$year <= 2070,]
    df$x <- as.numeric(df$x)
    df <- df[!is.na(df$x),]
    if (nrow(df) < 75) { next }
    
    # pick up model to be fitted depending on variable
    mdl <- tryCatch({
            do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = "gmst", lower = F)))
        }, error = function(cond) {
            return(NULL)
        })
    if (is.null(mdl)) { 
        mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = "gmst", lower = F, "method" = "Nelder-Mead")))
    }
    
    # use model's 2025 GMST to define factual climate        
    cov_now <- df[df$year == 2025,"gmst",drop = F]
    cov_hist <- cov_now - 1.3
    cov_fut <- cov_now + 1.3

    # bootstrap results
    res <- cmodel_results(mdl, rp = rp, cov_f = cov_now, cov_hist = cov_hist, cov_fut = cov_fut,
                          y_now = 2025, y_start = 1950, y_fut = 2070, nsamp = nsamp)
    res <- cbind(res, data.frame("ystart" = min(df$year), "yend" = max(df$year)))
    write.csv(res, res_fnm)

    # quick return level plots to check fit
    rlplot_fnm <- gsub("csv", "png", gsub("res.+/res", "mdl-eval/rlplot", res_fnm))
    if(!file.exists(rlplot_fnm)) {
        png(rlplot_fnm, h = 360, w = 480); par(cex.main = 1.4); {
            plot_returnlevels(mdl, cov_f = cov_now, cov_cf = cov_hist, nsamp = 100, main = gsub(".+/", "", gsub(".csv", "", fnm)))
        }; dev.off()
    }
}
print("Done.")

## HighResMIP

In [ ]:
for(fnm in list.files("ts-highresmip", pattern = varnm, full.names = T)) {
    
    gcm <- gsub(".+jjaso_|.dat","",fnm)
    print(gcm)
    
    res_fnm <- paste0("res-mdl-gmstonly/res-highresmip_",varnm,"_",gcm,".csv")
    if (file.exists(res_fnm)) next
    
    ts <- read.table(fnm, col.names = c("year", "x"))

    gmst_fnm <- list.files("~/00_WWA_project_folder/live/data/highresmip/GSAT", pattern = gcm, full.names = T)
    if(length(gmst_fnm) == 0) next
    if (!file.exists(gsub(".+/","ts-highresmip/",gmst_fnm))) file.copy(gmst_fnm, "ts-highresmip")
    
    gmst <- load_ts(gmst_fnm, col.names = c("year", "gmst"))
    
    df <- merge(ts, gmst)
    df$x <- as.numeric(df$x)
    df <- df[!is.na(df$x),]
    if (nrow(df) < 75) { next }
    
    # pick up model to be fitted depending on variable
    mdl <- tryCatch({
            do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = "gmst", lower = F)))
        }, error = function(cond) {
            return(NULL)
        })
    if (is.null(mdl)) { 
        mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = "gmst", lower = F, "method" = "Nelder-Mead")))
    }
    
    # use model's 2025 GMST to define factual climate        
    cov_now <- df[df$year == 2025,"gmst",drop = F]
    cov_hist <- cov_now - 1.3
    cov_fut <- cov_now + 1.3

    # bootstrap results
    res <- cmodel_results(mdl, rp = rp, cov_f = cov_now, cov_hist = cov_hist, cov_fut = cov_fut,
                          y_now = 2025, y_start = 1950, y_fut = 2070, nsamp = nsamp)
    res <- cbind(res, data.frame("ystart" = min(df$year), "yend" = max(df$year)))
    write.csv(res, res_fnm)

    # quick return level plots to check fit
    rlplot_fnm <- gsub("csv", "png", gsub("res.+/res", "mdl-eval/rlplot", res_fnm))
    if(!file.exists(rlplot_fnm)) {
        png(rlplot_fnm, h = 360, w = 480); par(cex.main = 1.4); {
            plot_returnlevels(mdl, cov_f = cov_now, cov_cf = cov_hist, nsamp = 100, main = gsub("ts/", "", gsub(".csv", "", fnm)))
        }; dev.off()
    }
}

## Compile models

In [ ]:
for (framing %in% c("cordex", "highresmip")) {
    mdl_res <- t(sapply(list.files("res-mdl-gmstonly", pattern = paste0(framing,".+",varnm), full.names = T), read.csv))
    rownames(mdl_res) <- apply(sapply(strsplit(gsub(".csv","",rownames(mdl_res)), "_"), "[", 3:6), 2, paste0, collapse = "_")
    write.csv(mdl_res, paste0("res-",framing,"_",varnm,"_gmst-only.csv"))
}

## Obs 1979-

In [2]:
gmst <- read.table("ts-obs/gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2025]

cov_2025 <- gmst[gmst$year == 2025,c("gmst"),drop = F]
cov_cf <- cov_2025 - 1.3

In [18]:
fl <- list.files("ts-obs", pattern = paste0("mexico-floods_",varnm), full.names = T)
for (fnm in fl) {

    ds_nm = gsub(".+jjaso_|.dat","",fnm)
    res_fnm = paste0("res-obs/res-1979_",varnm,"_gmst-only_",ds_nm,".csv")
    
    df <- merge(gmst, load_ts(fnm, col.names = c("year", "x")))
    df <- df[df$year >= 1979,]
    mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = "gmst", lower = F)))

    if (ds_nm %in% c("chirps", "mxhighresclimdb")) {
        rl_2025 <- eff_return_level(mdl, rp = rp, fixed_cov = cov_2025)
        mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = "gmst", lower = F,
                                                     ev = rl_2025, ev_year = 2025)))
    }

    if (file.exists(res_fnm)) next
    boot_res <- boot_ci(mdl, cov_f = cov_2025, cov_cf = cov_cf, nsamp = 1000)
    write.csv(boot_res, res_fnm)

    plot_fnm <- gsub("csv", "png", gsub("res-obs/res", "fig/trendplots", res_fnm))
    png(plot_fnm, h = png_res, w = 3 * png_res*1.5); {
        prep_window(c(1,3))
        plot_trend(mdl)
        plot_covtrend(mdl, "gmst")
        plot_returnlevels(mdl, cov_f = cov_2025, cov_cf = cov_cf[1,,drop = F], nsamp = 50, legend_pos = "bottomright")
    }; dev.off()
}

# GMST + Nino

In [7]:
# standardise observed Nino to have mean 0 and sd 1 over 1979-2023
nino_obs <- load_ts("ts-obs/mexico-floods_nino34det-sep.dat", col.names = c("year", "nino"))

nino_1979 <- nino_obs$nino[nino_obs$year >= 1979]
nino_std <- data.frame("year" = nino_obs$year, "nino" = (nino_obs$nino - mean(nino_1979)) / sd(nino_1979))

# use 2025 standardised Nino as covariate
nino_2025 <- nino_std$nino[nino_std$year == 2025]

rp <- switch(varnm, "rx3day" = 10, "pr-jjaso" = 10)

## Obs

In [3]:
gmst <- read.table("ts-obs/gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2025]

cov_df <- merge(gmst, nino_obs)

cov_2025 <- cov_df[cov_df$year == 2025,c("gmst", "nino"),drop = F]
cov_cf <- rbind("pi" = data.frame("gmst" = cov_2025$gmst - 1.3, "nino" = cov_2025$nino),
                "ninoneut" = data.frame("gmst" = cov_2025$gmst, "nino" = 0))

In [5]:
fl <- list.files("ts-obs", pattern = paste0("mexico-floods_",varnm), full.names = T)
for (fnm in fl) {

    ds_nm = gsub(".+jjaso_|.dat","",fnm)
    res_fnm = paste0("res-obs/res-obs_",varnm,"_gmst+nino_",ds_nm,".csv")
    
    df <- merge(cov_df, load_ts(fnm, col.names = c("year", "x")))
    mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = c("gmst", "nino"), lower = F)))

    if (ds_nm %in% c("chirps", "mxhighresclimdb")) {
        rl_2025 <- eff_return_level(mdl, rp = rp, fixed_cov = cov_2025)
        mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = c("gmst", "nino"), lower = F,
                                                     ev = rl_2025, ev_year = 2025)))
    }

    if (!file.exists(res_fnm)) {
        boot_res <- boot_ci(mdl, cov_f = cov_2025, cov_cf = cov_cf, nsamp = 1000)
        write.csv(boot_res, res_fnm)
    }

    plot_fnm <- gsub("csv", "png", gsub("res-obs/res-obs", "fig/trendplots", res_fnm))
    if (file.exists(plot_fnm)) next
    png(plot_fnm, h = 2 * png_res, w = 3 * png_res*1.5); {
        prep_window(c(2,3))
        plot_trend(mdl)
        plot_covtrend(mdl, "gmst")
        plot_covtrend(mdl, "nino")
        plot_returnlevels(mdl, cov_f = cov_2025, cov_cf = cov_cf[1,,drop = F], nsamp = 50, legend_pos = "bottomright")
        plot_returnlevels(mdl, cov_f = cov_2025, cov_cf = cov_cf[2,,drop = F], nsamp = 50, legend_pos = "bottomright")
    }; dev.off()
}

## CORDEX

In [ ]:
fl <- list.files("ts-cordex", pattern = paste0(varnm, "_CAM"), full.names = T)

for (fnm in fl) {
    print(fnm)

    if (fnm %in% c("ts-cordex/mexico-floods_rx3day_CAM-44_CNRM-CM5_r1i1p1_CRCM5.dat")) next
    
    res_fnm <- gsub(".dat", ".csv", gsub("ts-cordex/", "res-mdl-gmst+nino/res-cordex_", fnm))
    if (exists(res_fnm)) next

    gcm <- gsub("_", "_rcp85_", gsub("i1p1","",paste0(strsplit(fnm, "_")[[1]][4:5], collapse = "_")))
    
    nino_fnm <- list.files("ts-cordex", pattern = paste0("nino34det-sep_",gsub("_rcp85","",gcm)), full.names = T)
    if (length(nino_fnm) == 0) next

    gmst_fnm <- list.files("ts-cordex", paste0("gsat_",gcm, ".dat"), full.names = T)
    if (length(gmst_fnm) == 0) next

    nino <- load_ts(nino_fnm, col.names = c("year", "nino34"))
    gmst <- load_ts(gmst_fnm, col.names = c("year", "gmst"))
    ts <- load_ts(fnm, col.names = c("year", "pr"))
    df <- data.frame(apply(merge(merge(nino, gmst), ts), 2, as.numeric))

    # add standardised Nino covariate
    nino_clim <- df[(df$year >= 1979) & (df$year <= 2025), c("nino34", "pr")]
    df$nino_std <- (df$nino - mean(nino_clim$nino34)) / sd(nino_clim$nino34)

    # trend fitting
    mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "pr", "covnm" = c("gmst", "nino_std"), "lower" = F)))

    # use model's 2023 GMST & observed Nino to define factual climate
    gmst_2025 <- df$gmst[df$year == 2025]
    
    cov_2025 <- data.frame(gmst = gmst_2025, nino_std = nino_2025)
    cov_cf <- rbind("pi" = data.frame("gmst" = gmst_2025 - 1.3, "nino_std" = nino_2025), 
                    "neut" = data.frame("gmst" = gmst_2025, "nino_std" = 0))
    cov_fut <- data.frame(gmst = gmst_2025 + 1.3, nino_std = nino_2025)

    # bootstrap results
    res <- cmodel_results(mdl, rp = rp, cov_f = cov_2025, cov_hist = cov_cf, cov_fut = cov_fut,
                         y_now = 2025, y_start = 1979, y_fut = 2070, nsamp = nsamp)

    # bootstrap Nino amplitude & correlation between Nino & precip in this region
    set.seed(42)
    nino_boot <- sapply(1:nsamp, function(i) {
        boot_df <- nino_clim[sample(1:nrow(nino_clim), replace = T), ]
        c("nino_cor" = cor(boot_df$nino34, boot_df$pr), "nino_sd" = sd(boot_df$nino34))
    })
    nino_qq <- rbind("est" = c(cor(df$nino34, df$pr), sd(df$nino34)), apply(nino_boot,1,quantile,c(0.025, 0.975)))
    nino_qq <- t(data.frame(unlist(sapply(colnames(nino_qq), function(cnm) {
        setNames(nino_qq[,cnm], paste0(cnm,"_", c("est", "lower", "upper")))
    }, simplify = F, USE.NAMES = F))))
                                       
    res <- cbind(res, nino_qq)
    write.csv(res, res_fnm)

    # quick return level plots to check fit
    rlplot_fnm <- gsub("csv", "png", gsub("res-mdl-gmst+nino/res-cordex", "mdl-eval/rlplot-nino_", res_fnm))
    if(!file.exists(rlplot_fnm)) {
        png(rlplot_fnm, h = 480, w = 480 * 2); par(cex.main = 1.4, mfrow = c(1,2), oma = c(0,0,2,0)); {
            plot_returnlevels(mdl, cov_f = cov_2025, cov_cf = cov_cf["pi",,drop = F], nsamp = 100, main = "Present vs PI",legend_pos = "bottomright")
            plot_returnlevels(mdl, cov_f = cov_2025, cov_cf = cov_cf["neut",,drop = F], nsamp = 100, main ="Present vs neutral Nino3.4",legend_pos = "bottomright")
            mtext(outer = T, gsub(".dat","",paste0(strsplit(fnm, "_")[[1]][3:5], collapse = "_")), font = 2, cex = 1.5)
        }; dev.off()
    }
}

## HighResMIP

In [3]:
for(fnm in list.files("ts-highresmip", pattern = varnm, full.names = T)) {
    
    gcm <- gsub(".+jjaso_|.dat","",fnm)
    print(gcm)
    
    res_fnm <- gsub(".dat", ".csv", gsub("ts-highresmip/", "res-mdl-gmst+nino/res-highresmip_", fnm))
    # if (file.exists(res_fnm)) next

    gmst_fnm <- list.files("~/00_WWA_project_folder/live/data/highresmip/GSAT", pattern = gcm, full.names = T)
    if(length(gmst_fnm) == 0) next

    nino_fnm <- list.files("ts-highresmip", pattern = paste0("nino3.4-sep_HighResSST_",gcm), full.names = T)
    if (length(nino_fnm) == 0) next

    nino <- load_ts(nino_fnm, col.names = c("year", "nino34"))
    gmst <- load_ts(gmst_fnm, col.names = c("year", "gmst"))
    ts <- load_ts(fnm, col.names = c("year", "pr"))
    df <- data.frame(apply(merge(merge(nino, gmst), ts), 2, as.numeric))
        
    # add standardised Nino covariate
    nino_clim <- df[(df$year >= 1979) & (df$year <= 2025), c("nino34", "pr")]
    df$nino_std <- (df$nino - mean(nino_clim$nino34)) / sd(nino_clim$nino34)

    # trend fitting
    mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "pr", "covnm" = c("gmst", "nino_std"), "lower" = F)))
    
    # use model's 2023 GMST & observed Nino to define factual climate
    gmst_2025 <- df$gmst[df$year == 2025]
    
    cov_2025 <- data.frame(gmst = gmst_2025, nino_std = nino_2025)
    cov_cf <- rbind("pi" = data.frame("gmst" = gmst_2025 - 1.3, "nino_std" = nino_2025), 
                    "neut" = data.frame("gmst" = gmst_2025, "nino_std" = 0))
    cov_fut <- data.frame(gmst = gmst_2025 + 1.3, nino_std = nino_2025)

    # bootstrap results
    res <- cmodel_results(mdl, rp = rp, cov_f = cov_2025, cov_hist = cov_cf, cov_fut = cov_fut,
                         y_now = 2025, y_start = 1979, y_fut = 2070, nsamp = nsamp)

    # bootstrap Nino amplitude & correlation between Nino & precip in this region
    set.seed(42)
    nino_boot <- sapply(1:nsamp, function(i) {
        boot_df <- nino_clim[sample(1:nrow(nino_clim), replace = T), ]
        c("nino_cor" = cor(boot_df$nino34, boot_df$pr), "nino_sd" = sd(boot_df$nino34))
    })
    nino_qq <- rbind("est" = c(cor(df$nino34, df$pr), sd(df$nino34)), apply(nino_boot,1,quantile,c(0.025, 0.975)))
    nino_qq <- t(data.frame(unlist(sapply(colnames(nino_qq), function(cnm) {
        setNames(nino_qq[,cnm], paste0(cnm,"_", c("est", "lower", "upper")))
    }, simplify = F, USE.NAMES = F))))
                                       
    res <- cbind(res, nino_qq)
    write.csv(res, res_fnm)

        # quick return level plots to check fit
    rlplot_fnm <- gsub("csv", "png", gsub(".+/res-highresmip", "mdl-eval/rlplot-nino", res_fnm))
    if(!file.exists(rlplot_fnm)) {
        png(rlplot_fnm, h = 480, w = 480 * 2); par(cex.main = 1.4, mfrow = c(1,2), oma = c(0,0,2,0)); {
            plot_returnlevels(mdl, cov_f = cov_2025, cov_cf = cov_cf["pi",,drop = F], nsamp = 10, main = "Present vs PI",legend_pos = "bottomright")
            plot_returnlevels(mdl, cov_f = cov_2025, cov_cf = cov_cf["neut",,drop = F], nsamp = 10, main ="Present vs neutral Nino3.4",legend_pos = "bottomright")
            mtext(outer = T, gsub(".dat","",paste0(strsplit(fnm, "_")[[1]][3:5], collapse = "_")), font = 2, cex = 1.5)
        }; dev.off()
    }
}

[1] "CMCC-CM2-HR4_r1i1p1f1"
[1] "CMCC-CM2-VHR4_r1i1p1f1"
[1] "CNRM-CM6-1_r1i1p1f2"
[1] "CNRM-CM6-1-HR_r1i1p1f2"
[1] "EC-Earth3P_r1i1p1f1"
[1] "HadGEM3-GC31-HM_r1i1p1f1"
[1] "HadGEM3-GC31-LM_r1i14p1f1"
[1] "HadGEM3-GC31-MM_r1i1p1f1"
[1] "MPI-ESM1-2-HR_r1i1p1f1"
[1] "MPI-ESM1-2-XR_r1i1p1f1"


## Obs 1979-

In [20]:
# standardise observed Nino to have mean 0 and sd 1 over 1979-2023
nino_obs <- load_ts("ts-obs/mexico-floods_nino34det-sep.dat", col.names = c("year", "nino"))

nino_1979 <- nino_obs$nino[nino_obs$year >= 1979]
nino_std <- data.frame("year" = nino_obs$year, "nino" = (nino_obs$nino - mean(nino_1979)) / sd(nino_1979))

# use 2025 standardised Nino as covariate
nino_2025 <- nino_std$nino[nino_std$year == 2025]

rp <- switch(varnm, "rx3day" = 10, "pr-jjaso" = 10)

In [21]:
gmst <- read.table("ts-obs/gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2025]

cov_df <- merge(gmst, nino_obs)

cov_2025 <- cov_df[cov_df$year == 2025,c("gmst", "nino"),drop = F]
cov_cf <- rbind("pi" = data.frame("gmst" = cov_2025$gmst - 1.3, "nino" = cov_2025$nino),
                "ninoneut" = data.frame("gmst" = cov_2025$gmst, "nino" = 0))

In [22]:
fl <- list.files("ts-obs", pattern = paste0("mexico-floods_",varnm), full.names = T)

In [23]:
for (fnm in fl) {

    ds_nm = gsub(".+jjaso_|.dat","",fnm)
    res_fnm = paste0("res-obs/res-1979_",varnm,"_gmst+nino_",ds_nm,".csv")
    
    df <- merge(cov_df, load_ts(fnm, col.names = c("year", "x")))
    df <- df[df$year >= 1979,]
    mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = c("gmst", "nino"), lower = F)))

    if (ds_nm %in% c("chirps", "mxhighresclimdb")) {
        rl_2025 <- eff_return_level(mdl, rp = rp, fixed_cov = cov_2025)
        mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = c("gmst", "nino"), lower = F,
                                                     ev = rl_2025, ev_year = 2025)))
    }

    if (!file.exists(res_fnm)) {
        boot_res <- boot_ci(mdl, cov_f = cov_2025, cov_cf = cov_cf, nsamp = 1000)
        write.csv(boot_res, res_fnm)
    }

    plot_fnm <- gsub("csv", "png", gsub("res-obs/res", "fig/trendplots", res_fnm))
    if (file.exists(plot_fnm)) next
    png(plot_fnm, h = 2 * png_res, w = 3 * png_res*1.5); {
        prep_window(c(2,3))
        plot_trend(mdl)
        plot_covtrend(mdl, "gmst")
        plot_covtrend(mdl, "nino")
        plot_returnlevels(mdl, cov_f = cov_2025, cov_cf = cov_cf[1,,drop = F], nsamp = 50, legend_pos = "bottomright")
        plot_returnlevels(mdl, cov_f = cov_2025, cov_cf = cov_cf[2,,drop = F], nsamp = 50, legend_pos = "bottomright")
    }; dev.off()
}

Warning message in file(file, ifelse(append, "a", "w")):
“cannot open file 'res-obs/res-1979_rx3day_gmst+nino_ts-obs/mexico-floods_rx3day_chirps.csv': No such file or directory”


ERROR: Error in file(file, ifelse(append, "a", "w")): cannot open the connection


# Check trends pre- & post-1979

In [213]:
suppressMessages(library(rwwa))

# load covariates
nino <- load_ts("ts-obs/mexico-floods_nino34det-sep.dat", col.names = c("year", "nino"))

gmst <- read.table("ts-obs/gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2025]

cov_df <- merge(gmst, nino)

In [214]:
cov_2025 <- cov_df[cov_df$year == 2025,c("gmst", "nino"),drop = F]
cov_cf <- rbind("pi" = data.frame("gmst" = cov_2025$gmst - 1.3, "nino" = cov_2025$nino),
                "neut" = data.frame("gmst" = cov_2025$gmst, "nino" = 0))

In [216]:
varnm <- "rx3day"

df_all <- Reduce(function(x, y) merge(x, y, all=TRUE), 
                 sapply(list.files("ts-obs", pattern = varnm, full.names = T), 
                        function(fnm) load_ts(fnm, col.names = c("year", gsub(paste0(".+",varnm,"_|.dat"),"",fnm))), 
                        simplify = F))

In [ ]:
df_all <- merge(cov_df, df_all)
if (varnm == "pr-jjaso") colnames(df_all)[which(colnames(df_all) == "mxhighresclimdb")] <- "MxHRC"

mdl_args <- list("rx3day" = list(dist = "gev", type = "fixeddisp"),
                 "pr-jjaso" = list(dist = "norm", type = "fixeddisp"))[[varnm]]

In [220]:
res_gmst_alldata <- abind::abind(sapply(colnames(df_all)[-(1:3)], function(cnm) {
    df <- df_all[,c("year", "gmst", "nino", cnm)]
    df <- df[!is.na(df[,cnm]),]

    mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = cnm, covnm = "gmst", lower = F)))
    boot_ci(mdl, cov_f = cov_2025, cov_cf = cov_cf, nsamp = 100)
}, simplify = F), along = 0)

res_gmst_1979 <- abind::abind(sapply(colnames(df_all)[-(1:3)], function(cnm) {
    df <- df_all[,c("year", "gmst", "nino", cnm)]
    df <- df[!is.na(df[,cnm]),]
    df <- df[df$year >= 1979,]

    mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = cnm, covnm = "gmst", lower = F)))
    boot_ci(mdl, cov_f = cov_2025, cov_cf = cov_cf, nsamp = 100)
}, simplify = F), along = 0)

res_both_alldata <- abind::abind(sapply(colnames(df_all)[-(1:3)], function(cnm) {
    df <- df_all[,c("year", "gmst", "nino", cnm)]
    df <- df[!is.na(df[,cnm]),]

    mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = cnm, covnm = c("gmst", "nino"), lower = F)))
    boot_ci(mdl, cov_f = cov_2025, cov_cf = cov_cf, nsamp = 100)
}, simplify = F), along = 0)
                                 
res_both_1979 <- abind::abind(sapply(colnames(df_all)[-(1:3)], function(cnm) {
    df <- df_all[,c("year", "gmst", "nino", cnm)]
    df <- df[!is.na(df[,cnm]),]
    df <- df[df$year >= 1979,]

    mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = cnm, covnm = c("gmst", "nino"), lower = F)))
    boot_ci(mdl, cov_f = cov_2025, cov_cf = cov_cf, nsamp = 100)
}, simplify = F), along = 0)

# also check last 25 years
res_gmst_2000 <- abind::abind(sapply(colnames(df_all)[-(1:3)], function(cnm) {
    df <- df_all[,c("year", "gmst", "nino", cnm)]
    df <- df[!is.na(df[,cnm]),]
    df <- df[df$year >= 2000,]

    mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = cnm, covnm = "gmst", lower = F)))
    boot_ci(mdl, cov_f = cov_2025, cov_cf = cov_cf, nsamp = 100)
}, simplify = F), along = 0)
                                 
res_both_2000 <- abind::abind(sapply(colnames(df_all)[-(1:3)], function(cnm) {
    df <- df_all[,c("year", "gmst", "nino", cnm)]
    df <- df[!is.na(df[,cnm]),]
    df <- df[df$year >= 2000,]

    mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = cnm, covnm = c("gmst", "nino"), lower = F)))
    boot_ci(mdl, cov_f = cov_2025, cov_cf = cov_cf, nsamp = 100)
}, simplify = F), along = 0)

In [221]:
mlist <- list("GMST only, all data" = res_gmst_alldata,
              "GMST only, 1979 onwards" = res_gmst_1979,
              "GMST only, 2000 onwards" = res_gmst_2000,
              "GMST + ENSO, all data" = res_both_alldata,
              "GMST + ENSO, 1979 onwards" = res_both_1979,
              "GMST + ENSO, 2000 onwards" = res_both_2000)

In [222]:
save(mlist, file = paste0("tmp_boot-qq_compare-periods_", varnm, "_boot100.Rdata"))

In [223]:
plot_pars <-  c("dI_rel_pi", "dI_rel_neut")

png(paste0("fig/check-trends-per-model_",varnm,".png"), height = 360, width = 360*3); {
    prep_window(c(1,length(plot_pars)), h = 4.5, w = 6)

    for (r in plot_pars) {
        res <- abind::abind(sapply(mlist, "[",,r,, simplify = F), along = 0)
        nobs <- ncol(res)
    
        plot(0, type = "n", xlim = c(-1,(nobs+1)*6)-1, ylim = range(pretty(res)), main = r, xaxt = "n", xlab = "", ylab = "")
        cols <- c("orange", "darkgoldenrod2", "gold", "forestgreen", "chartreuse3", "yellowgreen")
    
        for (i in 1:nobs) {
            xx = 7*(i-1) + (1:6)
            segments(x0 = xx, y0 = res[,i,"2.5%"], y1 = res[,i,"97.5%"], lwd = 8, col = adjustcolor(cols, alpha = 0.4))
            points(xx, res[,i,"est"], pch = 21, bg = cols)
            }
        axis(1, at = (1:nobs - 0.5) * 7, colnames(res))
        abline(h = 0, lty = 2)
    }
mtext(varnm, outer = T, line = -2, side = 3, font = 2, cex = 1.5)
legend("bottomleft", fill = adjustcolor(cols, alpha = 0.4), legend = rownames(res), bty = "n", ncol = 2)
}; dev.off()

agg_record_1073919664 
                    2